In [2]:
from flask import Flask, render_template, jsonify,send_file
import pandas as pd
import warnings
import math
import numpy as np
from sqlalchemy import create_engine, text
from io import BytesIO
import psycopg2


color_scheme = {
  '1':  '#B83911',
  '10': 
     '#40916C',
  '11': 
     '#8C5CB7',
  
  '12': 
     '#C8C832',
  
  '14': 
     '#74C69D',
  
  '17': 
     '#0F7512',
  
  '18': 
     '#28C92B',
  
  '19': 
     'black',
  
  '2': 
     '#DFEC32',
  
  '20': 
     '#DF111C',
  
  '21': 
     '#A6AA22',
  
  '22': 
     '#C8C832',
  
  '23': 
     '#4B9CD3',
  
  '24': 
     '#2989D8',
  
  '3': 
     '#865B33',
  '4': 
     '#52B788',
  '41': 
     '#95D5B2',
  '42': 
     '#C93E17',
  '43': 
     '#29E6E4',
  '45': 
     '#FF00EA',
  '47': 
     '#DA9A51',
  '48': 
     '#2666B3',
  '5': 
     '#2666B3',
  '54': 
     '#28C92B',
  '6': 
     '#15831A',
  '67': 
     '#FF7516',
  '8': 
     '#BB8348',
  '88': 
     '#2F7DDE',
  '9': 
     '#2C89E9',
  '95': 
     '#E37823',
  '99': 
     '#BA702B',
   '15': '#646464',
   '51': '#646464',
   '34': '#646464',
   '62': '#646464',
   '7': '#646464',
   '16':'#646464',
   '31':'#646464', '38':'#646464', '77':'#646464', '78':'#646464'}


In [7]:
import os
os.listdir('../')

['23XI-Data-Science',
 '252024-notes.txt',
 'Abandoned Projects',
 'aws-23XI-data-science',
 'Balance',
 'Blue Marlin',
 'Brainstorming',
 'Completed Projects',
 'Data for DB',
 'demo notes.txt',
 'desktop.ini',
 'Docker Desktop.lnk',
 'flights',
 'img2df',
 'Output',
 'Restarts',
 'smt-win-ChicagoStreet_Version-6.0.2_Build-249',
 'SQLWorkbench - Shortcut.lnk',
 'TeamAnalytics - Shortcut.lnk',
 'Visual Studio Code.lnk',
 'WIP',
 'Workbench-Build129-with-optional-libs']

FileNotFoundError: [Errno 2] No such file or directory: '../Restarts/24LAC1 Drier Ratings.xlsx'